 ### Existing code from the online assignment

In [1]:
import os
import random
import re
import sys

DAMPING = 0.85
SAMPLES = 10000

In [2]:
def crawl(directory):
    """
    Parse a directory of HTML pages and check for links to other pages.
    Return a dictionary where each key is a page, and values are
    a list of all other pages in the corpus that are linked to by the page.
    """
    pages = dict()

    # Extract all links from HTML files
    for filename in os.listdir(directory):
        if not filename.endswith(".html"):
            continue
        with open(os.path.join(directory, filename)) as f:
            contents = f.read()
            links = re.findall(r"<a\s+(?:[^>]*?)href=\"([^\"]*)\"", contents)
            pages[filename] = set(links) - {filename}

    # Only include links to other pages in the corpus
    for filename in pages:
        pages[filename] = set(
            link for link in pages[filename]
            if link in pages
        )

    return pages

In [29]:
def transition_model(corpus, page, damping_factor):
    """
    Return a probability distribution over which page to visit next,
    given a current page.

    With probability `damping_factor`, choose a link at random
    linked to by `page`. With probability `1 - damping_factor`, choose
    a link at random chosen from all pages in the corpus.
    """
    
    m = len(corpus[page])
    n = len(corpus)
    p = 1 - damping_factor
    model = {}
    
    for page in corpus.keys():
        model[page] = p * 1/n
    
    for link in corpus[page]:
        model[link] += damping_factor*1/m
    
    return model

In [34]:
def sample_pagerank(corpus, damping_factor, n):
    """
    Return PageRank values for each page by sampling `n` pages
    according to transition model, starting with a page at random.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """

    num_pages = len(corpus)
    page_names = corpus.keys()
    
    
    first_page = random.randint(0, num_pages)
    print(first_page)
    sample = transition_model(corpus, first_page, damping_factor)
    
    for _ in range(n-1):
        rand_page = random.choice(sample.keys(), 1, sample.values())
        sample = transition_model(corpus, rand_page, damping_factor)
        
    return sample

In [10]:
def iterate_pagerank(corpus, damping_factor):
    """
    Return PageRank values for each page by iteratively updating
    PageRank values until convergence.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    raise NotImplementedError

In [35]:
def main():

    corpus = crawl("testdirectory/corpus3")
    #print(corpus)
    #print(transition_model(corpus, '1.html', DAMPING))
    ranks = sample_pagerank(corpus, DAMPING, SAMPLES)
    print(f"PageRank Results from Sampling (n = {SAMPLES})")
    # for page in sorted(ranks):
    #     print(f"  {page}: {ranks[page]:.4f}")
    # ranks = iterate_pagerank(corpus, DAMPING)
    # print(f"PageRank Results from Iteration")
    # for page in sorted(ranks):
    #     print(f"  {page}: {ranks[page]:.4f}")

main()

KeyError: 0

In [31]:
os.listdir("testdirectory/corpus2")

['python.html',
 'c.html',
 'logic.html',
 'programming.html',
 'inference.html',
 'algorithms.html',
 'recursion.html',
 'ai.html']